In [2]:
import scipy.stats as stats
import numpy as np
import math
import pandas as pd


In [18]:
cars_df = pd.read_csv('data/cars.csv')
cars_df = cars_df[cars_df['horsepower'] != '?']
cars_df.sample(20)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin,car_name
53,31.0,4,71.0,65.00,1773.0,19.0,71,3,toyota corolla 1200
321,32.2,4,108.0,75.00,2265.0,15.2,80,3,toyota corolla
86,14.0,8,304.0,150.0,3672.0,11.5,73,1,amc matador
200,18.0,6,250.0,78.00,3574.0,21.0,76,1,ford granada ghia
360,30.7,6,145.0,76.00,3160.0,19.6,81,2,volvo diesel
77,22.0,4,121.0,76.00,2511.0,18.0,72,2,volkswagen 411 (sw)
346,32.3,4,97.0,67.00,2065.0,17.8,81,3,subaru
28,9.0,8,304.0,193.0,4732.0,18.5,70,1,hi 1200d
320,37.0,4,119.0,92.00,2434.0,15.0,80,3,datsun 510 hatchback
355,33.7,4,107.0,75.00,2210.0,14.4,81,3,honda prelude


- Columns: MPG, Cylinders, Displacement, Weight, Acceleration, Model, Origin, Car Name
- 398 samples, no missing data, all data was numeric except Car Name
- 6 rows had '?' as the value for horsepower, those rows were removed

- Interested in difference in MPG or US-made and Japanese-made 4-cylinder vehicles

In [ ]:
# cars_df_sorted = cars_df[cars_df['cylinders'] == 4].groupby(cars_df['origin']).count()
# cars_df_sorted.head()


# creating new df with only American and Japanese manufactured vehicles that are 4-cylinders

cars_df = cars_df[(cars_df['origin'] != 2) & (cars_df['cylinders'] == 4)]
cars_df
# cars_df.groupby(cars_df['origin']).mean()

In [77]:
def sample_calcs(col1, col2='origin'):
    
    col1_df = pd.DataFrame()
    col1_df['Mean'] = cars_df.groupby(col2)[col1].mean()
    col1_df['STD'] = cars_df.groupby(col2)[col1].std()
    col1_df['Sample Size'] = cars_df.groupby(col2)[col1].count()

    return col1_df


,Mean,STD,Sample Size
origin,,,
1,28.013043,4.566596,69
3,31.595652,5.435787,69


In [ ]:
def graph_samp_dist(mean, std, sample_size, color):

    std_error = std / np.sqrt(sample_size)
    sample_dist_mean = stats.norm(mean, std_error)  
    xs = np.linspace(mean-(4*std_error), mean+(4*std_error), 300)

    pdfs = sample_dist_mean.pdf(xs)

    ax.plot(xs, pdfs, color=color)


comparison_df = sample_calcs('mpg')
graph_samp_dist(comparison_df)